##### Scraping Website with BeautifulSoup

In [14]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

r = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") 
soup = BeautifulSoup(r.content, "html.parser")

table = soup.find(class_='wikitable sortable').find_all("tr")

postal_codes = []

for tr in table:
    row = [ np.nan if tr.text.strip() == 'Not assigned' else tr.text.strip() for tr in tr.find_all('td') ]
    postal_codes.append(row)
    
postal_codes = list(filter(lambda x : len(x) == 3, postal_codes) )
postal_codes_df = pd.DataFrame(postal_codes, columns= [ "Postal Code", "Borough", "Neighborhood"])
postal_codes_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


##### Data pre-processing 

In [24]:
# Drop NaN
postal_codes_df = postal_codes_df[ postal_codes_df['Borough'].notna() ]
# Reset Index
postal_codes_df = postal_codes_df.reset_index(drop=True)
# If Neighborhood is Nan, fill with Borough
postal_codes_df.Neighborhood.fillna(postal_codes_df.Borough, inplace=True)

postal_codes_df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [35]:
postal_codes_df.shape

(103, 3)

In [27]:
# Get lat long
lat_long_df = pd.read_csv("https://cocl.us/Geospatial_data")

In [34]:
# Join Postal Code with LatLong
geo = postal_codes_df.set_index('Postal Code').join( lat_long_df.set_index('Postal Code') ).reset_index()
geo

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [33]:
geo.shape

(103, 5)